In [ ]:
import cv2
import numpy as np

def compute_smoke_mask(normal_image, fire_image):
    normal_gray = cv2.cvtColor(normal_image, cv2.COLOR_BGR2GRAY)
    fire_gray = cv2.cvtColor(fire_image, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(fire_gray, normal_gray)
    _, smoke_mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    smoke_mask = cv2.GaussianBlur(smoke_mask, (51, 51), 0)
    return smoke_mask

def blend_smoke(normal_image, fire_image, smoke_mask):
    smoke_mask_3channel = cv2.merge([smoke_mask, smoke_mask, smoke_mask]) / 255.0
    smoke = cv2.bitwise_and(fire_image, fire_image, mask=smoke_mask)
    blended_image = (normal_image.astype(float) * (1 - smoke_mask_3channel) +
                     smoke.astype(float) * smoke_mask_3channel).astype(np.uint8)
    return blended_image

normal_image = cv2.imread('no_fire_image_path.png')
fire_image = cv2.imread('fire_image_path.png')

if normal_image is None:
    raise FileNotFoundError("Error: 'nofire.png' not found or could not be loaded.")
if fire_image is None:
    raise FileNotFoundError("Error: 'fire.png' not found or could not be loaded.")

fire_image_resized = cv2.resize(fire_image, (normal_image.shape[1], normal_image.shape[0]))
smoke_mask = compute_smoke_mask(normal_image, fire_image_resized)
cv2.imwrite('smoke_mask_debug.png', smoke_mask)
result_image = blend_smoke(normal_image, fire_image_resized, smoke_mask)
cv2.imwrite('output_image_name.png', result_image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imshow('Smoke Mask', smoke_mask)
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
